In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
bc = datasets.load_breast_cancer()

In [3]:
X,y  = bc.data,bc.target

In [4]:
n_sample,n_features = X.shape

In [5]:
print(n_sample,n_features)

569 30


In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1234)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [7]:
y_train.shape

torch.Size([455, 1])

In [8]:
# model (f = wx+b,then sigmoid at end)
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear = nn.Linear(n_input_features,1)
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [9]:
model = LogisticRegression(n_features)

In [10]:
#loss
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [11]:
#training
num_epoch = 100
for epoch in range(num_epoch):
    y_predicted = model(X_train)
    loss = criterion(y_predicted,y_train)
    #back pass
    loss.backward()
    # update
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 1 == 0:
        print(f'epoch:{epoch+1},loss={loss.item():.4f}')

epoch:1,loss=0.9554
epoch:2,loss=0.9230
epoch:3,loss=0.8923
epoch:4,loss=0.8633
epoch:5,loss=0.8358
epoch:6,loss=0.8099
epoch:7,loss=0.7854
epoch:8,loss=0.7624
epoch:9,loss=0.7407
epoch:10,loss=0.7203
epoch:11,loss=0.7011
epoch:12,loss=0.6830
epoch:13,loss=0.6660
epoch:14,loss=0.6499
epoch:15,loss=0.6348
epoch:16,loss=0.6205
epoch:17,loss=0.6070
epoch:18,loss=0.5942
epoch:19,loss=0.5821
epoch:20,loss=0.5707
epoch:21,loss=0.5598
epoch:22,loss=0.5495
epoch:23,loss=0.5396
epoch:24,loss=0.5303
epoch:25,loss=0.5214
epoch:26,loss=0.5128
epoch:27,loss=0.5047
epoch:28,loss=0.4969
epoch:29,loss=0.4894
epoch:30,loss=0.4823
epoch:31,loss=0.4754
epoch:32,loss=0.4688
epoch:33,loss=0.4625
epoch:34,loss=0.4563
epoch:35,loss=0.4505
epoch:36,loss=0.4448
epoch:37,loss=0.4393
epoch:38,loss=0.4340
epoch:39,loss=0.4289
epoch:40,loss=0.4240
epoch:41,loss=0.4192
epoch:42,loss=0.4146
epoch:43,loss=0.4101
epoch:44,loss=0.4057
epoch:45,loss=0.4015
epoch:46,loss=0.3974
epoch:47,loss=0.3935
epoch:48,loss=0.3896
e

In [12]:
#testing

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])

print(f'accuracy: = {acc:.4f}')

accuracy: = 0.9298
